In [4]:
!pip install openpyxl
import pandas as pd
import random

# Create a list to hold employee data
employees = []

# List of realistic first and last names
first_names = ['James', 'Mary', 'John', 'Patricia', 'Robert', 'Jennifer', 'Michael', 'Linda',
               'William', 'Elizabeth', 'David', 'Susan', 'Richard', 'Jessica', 'Joseph',
               'Sarah', 'Thomas', 'Karen', 'Charles', 'Nancy', 'Christopher', 'Lisa',
               'Daniel', 'Margaret', 'Matthew']

last_names = ['Smith', 'Johnson', 'Williams', 'Jones', 'Brown', 'Davis', 'Miller', 'Wilson',
              'Moore', 'Taylor', 'Anderson', 'Thomas', 'Jackson', 'White', 'Harris', 'Martin',
              'Thompson', 'Garcia', 'Martinez', 'Robinson', 'Clark', 'Rodriguez', 'Lewis',
              'Lee', 'Walker']

# Set random seed for reproducibility
random.seed(42)
# Shuffle the names to create random combinations
random.shuffle(first_names)
random.shuffle(last_names)

# Generate 20 full-time employees
for i in range(20):
    employees.append({
        'employee_name': f"{first_names[i]} {last_names[i]}",
        'contract': 'full-time',
        'hours_per_month': 140
    })

# Generate 5 part-time employees
for i in range(20, 25):
    employees.append({
        'employee_name': f"{first_names[i]} {last_names[i]}",
        'contract': 'part-time',
        'hours_per_month': 80
    })

# Create a DataFrame
df = pd.DataFrame(employees)

# Save to Excel file
df.to_excel("employees.xlsx", index=False)

print("Excel file 'employees.xlsx' created with the following data:")
print(df.to_string())
print(f"\nTotal: {len(df)} employees")
print(f"Full-time: {len(df[df['contract'] == 'full-time'])} employees")
print(f"Part-time: {len(df[df['contract'] == 'part-time'])} employees")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 4.5 MB/s eta 0:00:00
Excel file 'employees.xlsx' created with the following data:
           employee_name   contract  hours_per_month
0        Thomas Williams  full-time              140
1        Richard Jackson  full-time              140
2       Elizabeth Thomas  full-time              140
3           Nancy Wilson  full-time              140
4         Charles Walker  full-time              140
5          Michael Jones  full-time              140
6        Jennifer Taylor  full-time              140
7            David Lewis  full-time              140
8           Sarah Garcia  full-time              140
9            Lisa Martin  full-time              140
10       Susan Rodriguez  full-time              140
11           Karen Clark  full-time              140
12         Mary Thompson  full-time              140
13         Joseph Harris  full-time              140
14        Daniel Johnson  full-time              140
15      Jes

In [7]:
!pip install ortools
from ortools.sat.python import cp_model
import pandas as pd
import openpyxl
import datetime
from openpyxl.styles import Alignment, PatternFill, Font, Border, Side
from openpyxl.utils import get_column_letter
from openpyxl.chart import PieChart, Reference

def create_schedule(employee_excel_path, restaurant_data, labor_laws):
    # Load employee data from Excel
    try:
        df = pd.read_excel(employee_excel_path)
        print(f"Successfully loaded employee data with {len(df)} employees")
    except Exception as e:
        print(f"Error loading Excel file: {e}")
        return None

    # Convert DataFrame to employee data structure
    employee_data = []
    for i, row in df.iterrows():
        employee = {
            'id': i + 1,  # Create sequential IDs
            'name': row['employee_name'],
            'contract': row['contract'],
            'max_hours': row['hours_per_month'],
            'wage_rate': 15,  # Default wage rate
            'unavailable_days': [],  # Default to available all days
        }

        # Different defaults based on contract type
        if row['contract'].lower() == 'full-time':
            employee['wage_rate'] = 18  # Higher wage for full-time
        elif row['contract'].lower() == 'part-time':
            employee['wage_rate'] = 15  # Lower wage for part-time

        employee_data.append(employee)

    # Create the CP-SAT model
    model = cp_model.CpModel()

    # Create variables for each employee and each shift
    shifts = {}
    for employee in employee_data:
        for day in range(30):  # Assuming a monthly schedule
            for time_slot in range(len(restaurant_data['time_slots'])):
                shifts[(employee['id'], day, time_slot)] = model.NewBoolVar(
                    f'shift_{employee["id"]}_{day}_{time_slot}')

    # Add constraints
    for employee in employee_data:
        # Maximum working hours constraint (from Excel)
        model.Add(sum(shifts[(employee['id'], d, t)] * restaurant_data['time_slot_duration'][t]
                    for d in range(30)
                    for t in range(len(restaurant_data['time_slots'])))
                 <= employee['max_hours'])

        # Limit to 1 shift per day for each employee
        for day in range(30):
            model.Add(sum(shifts[(employee['id'], day, t)]
                        for t in range(len(restaurant_data['time_slots'])))
                     <= 1)

        # Contract-specific constraints - SIGNIFICANTLY RELAXED
        if employee['contract'].lower() == 'full-time':
            # Full-time employees should work at least a minimum number of hours - RELAXED to 60%
            min_hours = int(employee['max_hours'] * 0.60)  # Reduced to 60% of max for feasibility
            model.Add(sum(shifts[(employee['id'], d, t)] * restaurant_data['time_slot_duration'][t]
                        for d in range(30)
                        for t in range(len(restaurant_data['time_slots'])))
                     >= min_hours)

            # Full-time employees work at least 2 days per week (adjusted from 4 to 2)
            for week in range(4):  # Assuming 4 weeks in a month
                start_day = week * 7
                end_day = start_day + 7
                if start_day < 30:  # Make sure we're not out of bounds
                    model.Add(sum(shifts[(employee['id'], d, t)]
                                for d in range(start_day, min(end_day, 30))
                                for t in range(len(restaurant_data['time_slots'])))
                             >= 2)  # Reduced from 4 to 2

    # Minimum daily staffing requirements - total of at least 16 staff per day
    for day in range(30):
        # Calculate total staff for the day across all time slots
        total_staff_day = sum(shifts[(employee['id'], day, t)]
                             for employee in employee_data
                             for t in range(len(restaurant_data['time_slots'])))

        # Require at least 16 total staff per day
        model.Add(total_staff_day >= restaurant_data['min_daily_staff'])

    # Minimum staffing requirements per time slot
    if 'min_staff' in restaurant_data:
        for day in range(30):
            for t in range(len(restaurant_data['time_slots'])):
                # Ensure minimum staff per shift
                model.Add(sum(shifts[(employee['id'], day, t)]
                            for employee in employee_data)
                         >= restaurant_data['min_staff'][t])

    # Add objectives
    objective_terms = []

    # Minimize Labor Costs
    for employee in employee_data:
        cost_term = sum(shifts[(employee['id'], d, t)] * employee['wage_rate'] *
                      restaurant_data['time_slot_duration'][t]
                     for d in range(30)
                     for t in range(len(restaurant_data['time_slots'])))
        objective_terms.append(cost_term)

    # Weekend preference for full-time employees - REDUCED WEIGHT
    weekend_weight = 20  # Reduced weight to allow more flexibility
    for employee in employee_data:
        if employee['contract'].lower() == 'full-time':
            for day in range(30):
                # Check if this is a weekend
                day_of_week = (day % 7)
                if day_of_week == 5 or day_of_week == 6:  # Weekend days
                    # Give preference to not working on weekends for full-time employees
                    for t in range(len(restaurant_data['time_slots'])):
                        objective_terms.append(weekend_weight * shifts[(employee['id'], day, t)])

    # Set the objective function
    model.Minimize(sum(objective_terms))

    # Solve the model
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 300.0  # Keep 5 minutes time limit
    print("Solving scheduling problem...")
    status = solver.Solve(model)

    # Extract the schedule
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(f"Solution found with status: {status}")
        # Create a schedule dictionary
        schedule = {}
        for employee in employee_data:
            schedule[employee['id']] = []
            for day in range(30):
                daily_shifts = []
                for t in range(len(restaurant_data['time_slots'])):
                    if solver.Value(shifts[(employee['id'], day, t)]) == 1:
                        daily_shifts.append(restaurant_data['time_slots'][t])
                schedule[employee['id']].append(daily_shifts)

        # Calculate statistics
        employee_stats = []
        total_cost = 0

        for employee in employee_data:
            hours_worked = sum(solver.Value(shifts[(employee['id'], d, t)]) *
                              restaurant_data['time_slot_duration'][t]
                             for d in range(30)
                             for t in range(len(restaurant_data['time_slots'])))

            cost = hours_worked * employee['wage_rate']
            total_cost += cost

            days_worked = sum(1 for d in range(30)
                             if any(solver.Value(shifts[(employee['id'], d, t)]) == 1
                                   for t in range(len(restaurant_data['time_slots']))))

            employee_stats.append({
                'id': employee['id'],
                'name': employee['name'],
                'contract': employee['contract'],
                'hours_worked': hours_worked,
                'max_hours': employee['max_hours'],
                'hours_percentage': round(hours_worked / employee['max_hours'] * 100, 1),
                'days_worked': days_worked,
                'cost': cost
            })

        return {
            'schedule': schedule,
            'employee_stats': employee_stats,
            'total_cost': total_cost,
            'status': 'Optimal' if status == cp_model.OPTIMAL else 'Feasible'
        }
    else:
        print(f"No solution found. Status: {status}")
        return {
            'schedule': None,
            'status': 'Infeasible or time limit reached'
        }

def export_simple_schedule(result, employee_data, restaurant_data, output_path):
    """Export a simplified schedule to Excel with employee names, dates, shift times, and a manager dashboard"""
    if not result['schedule']:
        print("No schedule to export")
        return

    # Create a new workbook
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Monthly Schedule"

    # Formatting
    center_align = Alignment(horizontal='center', vertical='center')
    header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
    weekend_fill = PatternFill(start_color="FFD966", end_color="FFD966", fill_type="solid")
    header_font = Font(bold=True, color="FFFFFF")
    thin_border = Border(left=Side(style='thin'), right=Side(style='thin'),
                         top=Side(style='thin'), bottom=Side(style='thin'))

    # Get current month dates
    today = datetime.date.today()
    first_day = today.replace(day=1)

    # Title
    ws['A1'] = f"Monthly Schedule - {first_day.strftime('%B %Y')}"
    ws['A1'].font = Font(bold=True, size=14)
    ws.merge_cells('A1:AE1')  # Merge cells for title

    # Employee names column header
    ws['A3'] = "Employee"
    ws['A3'].font = header_font
    ws['A3'].fill = header_fill
    ws['A3'].alignment = center_align
    ws['A3'].border = thin_border

    # Set column width for employee names
    ws.column_dimensions['A'].width = 20

    # Create date headers
    day_row = 3
    for day in range(1, 32):  # Up to 31 days in a month
        try:
            date = first_day.replace(day=day)
            col_letter = get_column_letter(day + 1)

            # Date header
            ws[f'{col_letter}{day_row}'] = date.strftime('%a\n%d')
            ws[f'{col_letter}{day_row}'].font = header_font
            ws[f'{col_letter}{day_row}'].fill = header_fill
            ws[f'{col_letter}{day_row}'].alignment = center_align
            ws[f'{col_letter}{day_row}'].border = thin_border

            # Set column width for date columns
            ws.column_dimensions[col_letter].width = 15

        except ValueError:
            # Skip if the date is invalid (e.g., February 30)
            break

    # Fill in employee schedules
    row = 4
    for emp_stat in result['employee_stats']:
        # Employee name
        employee_id = emp_stat['id']
        ws[f'A{row}'] = emp_stat['name']
        ws[f'A{row}'].border = thin_border

        # Fill in shifts for each day
        for day in range(min(len(result['schedule'][employee_id]), 31)):
            col_letter = get_column_letter(day + 2)  # +2 because column A is for names and days start at 1

            # Get all shifts for this employee on this day
            daily_shifts = result['schedule'][employee_id][day]

            # Format shift times for display
            shift_text = ""
            if daily_shifts:
                formatted_shifts = []
                for shift in daily_shifts:
                    formatted_shifts.append(shift)
                shift_text = ", ".join(formatted_shifts)

            # Add to cell
            ws[f'{col_letter}{row}'] = shift_text
            ws[f'{col_letter}{row}'].border = thin_border
            ws[f'{col_letter}{row}'].alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

            # Increase row height to accommodate wrapped text
            if ws.row_dimensions[row].height is None or ws.row_dimensions[row].height < 30:
                ws.row_dimensions[row].height = 30

            # Weekend highlighting
            date = first_day.replace(day=day+1)
            if date.weekday() >= 5:  # Weekend
                ws[f'{col_letter}{row}'].fill = weekend_fill

        row += 1

    # Add a simple summary row at the bottom
    summary_row = row + 1
    ws[f'A{summary_row}'] = "Total Staff"
    ws[f'A{summary_row}'].font = Font(bold=True)
    ws[f'A{summary_row}'].border = thin_border

    # Count staff for each day
    for day in range(min(31, 30)):  # Up to 30 days or month end
        staff_count = sum(1 for emp_id in result['schedule']
                         if day < len(result['schedule'][emp_id]) and result['schedule'][emp_id][day])

        col_letter = get_column_letter(day + 2)
        ws[f'{col_letter}{summary_row}'] = staff_count
        ws[f'{col_letter}{summary_row}'].border = thin_border
        ws[f'{col_letter}{summary_row}'].alignment = center_align

    # ===== Create Manager Dashboard =====
    dashboard = wb.create_sheet("Manager Dashboard")

    # Title
    dashboard['A1'] = f"Manager Dashboard - Employee Hours for {first_day.strftime('%B %Y')}"
    dashboard['A1'].font = Font(bold=True, size=14)
    dashboard.merge_cells('A1:G1')

    # Section 1: Employee Hours Summary
    dashboard['A3'] = "Employee Hours Summary"
    dashboard['A3'].font = Font(bold=True, size=12)
    dashboard.merge_cells('A3:G3')

    # Headers
    headers = ["Employee", "Contract", "Hours Worked", "Target Hours", "Utilization", "Weekend Days", "Days Worked"]
    for col, header in enumerate(headers):
        cell = dashboard.cell(row=5, column=col+1)
        cell.value = header
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = center_align
        cell.border = thin_border
        dashboard.column_dimensions[get_column_letter(col+1)].width = 15

    # Data
    row = 6
    for stat in result['employee_stats']:
        # Count weekend days worked
        weekend_days = 0
        for day in range(30):
            day_of_week = day % 7
            if day_of_week == 5 or day_of_week == 6:  # Weekend (assuming 0=Monday)
                if day < len(result['schedule'][stat['id']]) and result['schedule'][stat['id']][day]:
                    weekend_days += 1

        # Calculate utilization percentage against target
        utilization = f"{stat['hours_percentage']}%"

        # Employee row
        dashboard.cell(row=row, column=1).value = stat['name']
        dashboard.cell(row=row, column=2).value = stat['contract']
        dashboard.cell(row=row, column=3).value = stat['hours_worked']
        dashboard.cell(row=row, column=4).value = stat['max_hours']
        dashboard.cell(row=row, column=5).value = utilization
        dashboard.cell(row=row, column=6).value = weekend_days
        dashboard.cell(row=row, column=7).value = stat['days_worked']

        # Apply formatting to all cells in the row
        for col in range(1, 8):
            cell = dashboard.cell(row=row, column=col)
            cell.border = thin_border
            cell.alignment = center_align

            # Highlight utilization based on targets - ADJUSTED THRESHOLDS
            if col == 5:  # Utilization column
                percentage = stat['hours_percentage']
                if percentage < 60:
                    # Understaffed - highlight red
                    cell.fill = PatternFill(start_color="FFCCCC", end_color="FFCCCC", fill_type="solid")
                elif percentage > 95:
                    # Overstaffed - highlight yellow
                    cell.fill = PatternFill(start_color="FFFFCC", end_color="FFFFCC", fill_type="solid")
                elif percentage >= 75 and percentage <= 95:
                    # Optimal - highlight green
                    cell.fill = PatternFill(start_color="CCFFCC", end_color="CCFFCC", fill_type="solid")

        row += 1

    # Add totals row
    dashboard.cell(row=row, column=1).value = "TOTALS"
    dashboard.cell(row=row, column=1).font = Font(bold=True)
    dashboard.cell(row=row, column=3).value = f"=SUM(C6:C{row-1})"
    dashboard.cell(row=row, column=4).value = f"=SUM(D6:D{row-1})"
    dashboard.cell(row=row, column=5).value = f"={round(100 * sum(stat['hours_worked'] for stat in result['employee_stats']) / sum(stat['max_hours'] for stat in result['employee_stats']), 1)}%"
    dashboard.cell(row=row, column=6).value = f"=SUM(F6:F{row-1})"
    dashboard.cell(row=row, column=7).value = f"=SUM(G6:G{row-1})"

    # Format totals row
    for col in range(1, 8):
        cell = dashboard.cell(row=row, column=col)
        cell.border = thin_border
        cell.alignment = center_align
        cell.font = Font(bold=True)

    # Section 2: Daily Staffing Overview
    row += 3
    dashboard.cell(row=row, column=1).value = "Daily Staffing Overview"
    dashboard.cell(row=row, column=1).font = Font(bold=True, size=12)
    dashboard.merge_cells(start_row=row, start_column=1, end_row=row, end_column=7)

    # Headers for daily staffing
    row += 2
    dashboard.cell(row=row, column=1).value = "Date"
    dashboard.cell(row=row, column=2).value = "Day"
    dashboard.cell(row=row, column=3).value = "Total Staff"
    dashboard.cell(row=row, column=4).value = "Full-Time"
    dashboard.cell(row=row, column=5).value = "Part-Time"
    dashboard.cell(row=row, column=6).value = "Total Hours"
    dashboard.cell(row=row, column=7).value = "Status"

    # Format headers
    for col in range(1, 8):
        cell = dashboard.cell(row=row, column=col)
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = center_align
        cell.border = thin_border

    # Calculate daily staffing
    days_in_month = (first_day.replace(month=first_day.month % 12 + 1, day=1) - datetime.timedelta(days=1)).day

    for day in range(days_in_month):
        row += 1
        date = first_day.replace(day=day+1)

        # Count staff by type
        full_time_count = 0
        part_time_count = 0
        total_hours = 0

        for emp_stat in result['employee_stats']:
            emp_id = emp_stat['id']
            if day < len(result['schedule'][emp_id]) and result['schedule'][emp_id][day]:
                if emp_stat['contract'].lower() == 'full-time':
                    full_time_count += 1
                else:
                    part_time_count += 1

                # Calculate hours for this day
                for shift in result['schedule'][emp_id][day]:
                    shift_index = restaurant_data['time_slots'].index(shift)
                    total_hours += restaurant_data['time_slot_duration'][shift_index]

        # Determine staffing status - ADJUSTED THRESHOLDS
        day_of_week = date.weekday()
        if day_of_week >= 5:  # Weekend
            if full_time_count + part_time_count < 8:
                status = "Understaffed"
                status_color = "FFCCCC"  # Light red
            elif full_time_count + part_time_count > 22:
                status = "Overstaffed"
                status_color = "FFFFCC"  # Light yellow
            else:
                status = "Optimal"
                status_color = "CCFFCC"  # Light green
        else:  # Weekday
            if full_time_count + part_time_count < 5:
                status = "Understaffed"
                status_color = "FFCCCC"  # Light red
            elif full_time_count + part_time_count > 20:
                status = "Overstaffed"
                status_color = "FFFFCC"  # Light yellow
            else:
                status = "Optimal"
                status_color = "CCFFCC"  # Light green

        # Fill in daily staffing data
        dashboard.cell(row=row, column=1).value = date.strftime("%m/%d/%Y")
        dashboard.cell(row=row, column=2).value = date.strftime("%A")
        dashboard.cell(row=row, column=3).value = full_time_count + part_time_count
        dashboard.cell(row=row, column=4).value = full_time_count
        dashboard.cell(row=row, column=5).value = part_time_count
        dashboard.cell(row=row, column=6).value = total_hours
        dashboard.cell(row=row, column=7).value = status

        # Format row
        for col in range(1, 8):
            cell = dashboard.cell(row=row, column=col)
            cell.border = thin_border
            cell.alignment = center_align

            # Highlight weekends
            if date.weekday() >= 5:
                cell.fill = weekend_fill

        # Status highlighting
        dashboard.cell(row=row, column=7).fill = PatternFill(
            start_color=status_color, end_color=status_color, fill_type="solid")

    # Save the workbook
    wb.save(output_path)
    print(f"Schedule with manager dashboard exported to {output_path}")

# Main execution function
def run_scheduler():
    print("Starting restaurant employee scheduler with 8-hour shifts")

    # Example Data Structures
    employee_excel_path = "employees.xlsx"  # Path to your Excel file

    # Check if employee file exists
    try:
        df = pd.read_excel(employee_excel_path)
        print(f"Using existing employee data with {len(df)} employees")
    except Exception as e:
        print(f"Error loading employee data: {e}")
        return False

    # Create 8-hour shift time slots from 4:00 to 23:30
    # Each shift is exactly 8 hours
    time_slots = [
        "04:00-12:00",  # Early morning (8 hours)
        "08:00-16:00",  # Morning to afternoon (8 hours)
        "12:00-20:00",  # Lunch to evening (8 hours)
        "15:30-23:30"   # Late shift (8 hours)
    ]

    restaurant_data = {
        'time_slots': time_slots,
        'time_slot_duration': [8, 8, 8, 8],  # All shifts are 8 hours
        'min_staff': [3, 6, 6, 6],  # Minimum staff per shift
        'min_daily_staff': 16,      # Total minimum staff per day
    }

    labor_laws = {
        'max_shifts_per_day': 1,  # Maximum 1 shift per day per employee
        'min_rest_hours': 10,     # Minimum 10 hours between shifts
        'enforce_fairness': False  # Turn off fairness to increase feasibility
    }

    print("Creating schedule with 8-hour shifts:")
    print(f"  Time slots: {restaurant_data['time_slots']}")
    print(f"  Minimum staff per shift: {restaurant_data['min_staff']}")
    print(f"  Minimum total staff per day: {restaurant_data['min_daily_staff']}")
    print(f"  Labor laws: {labor_laws}")
    print("This may take a few minutes...")

    # Attempt to create a schedule with these requirements
    result = create_schedule(employee_excel_path, restaurant_data, labor_laws)

    # If first attempt fails, try with relaxed constraints
    if not result or not result.get('schedule'):
        print("Initial scheduling attempt failed. Trying with relaxed constraints...")

        # Try with more relaxed constraints if the first attempt fails
        restaurant_data['min_staff'] = [2, 5, 5, 5]  # Slightly reduced minimums per shift
        restaurant_data['min_daily_staff'] = 14       # Reduced daily minimum

        # Try again with relaxed constraints
        result = create_schedule(employee_excel_path, restaurant_data, labor_laws)

        # If still fails, try one more time with very relaxed constraints
        if not result or not result.get('schedule'):
            print("Second attempt failed. Trying with further relaxed constraints...")
            restaurant_data['min_staff'] = [1, 3, 3, 3]  # Greatly reduced minimums per shift
            restaurant_data['min_daily_staff'] = 12       # Greatly reduced daily minimum
            result = create_schedule(employee_excel_path, restaurant_data, labor_laws)

    # Process the results
    if result and result.get('schedule'):
        print(f"Schedule created successfully. Status: {result['status']}")
        print(f"Total labor cost: ${result['total_cost']:.2f}")

        # Export to Excel format
        output_path = "restaurant_schedule.xlsx"
        export_simple_schedule(result,
                              [{'id': stat['id'], 'name': stat['name']} for stat in result['employee_stats']],
                              restaurant_data,
                              output_path)

        print(f"\nSchedule successfully exported to {output_path}")
        print("The file contains:")
        print("  1. Monthly Schedule - Main schedule with shifts for each employee")
        print("  2. Manager Dashboard - Employee hour summaries and daily staffing")

        # Print summary statistics
        total_hours = sum(stat['hours_worked'] for stat in result['employee_stats'])
        avg_utilization = sum(stat['hours_percentage'] for stat in result['employee_stats']) / len(result['employee_stats'])

        print("\nSummary Statistics:")
        print(f"  Total employees scheduled: {len(result['employee_stats'])}")
        print(f"  Total hours scheduled: {total_hours} hours")
        print(f"  Average employee utilization: {avg_utilization:.1f}%")
        print(f"  Average cost per hour: ${result['total_cost'] / total_hours if total_hours > 0 else 0:.2f}")

        # Analyze actual staffing per shift
        print("\nActual Daily Staffing Analysis:")
        shift_counts = {ts: [] for ts in restaurant_data['time_slots']}
        total_staff_per_day = []

        for day in range(30):
            day_shift_counts = {ts: 0 for ts in restaurant_data['time_slots']}
            day_total = 0

            for employee_id in result['schedule']:
                if day < len(result['schedule'][employee_id]):
                    for shift in result['schedule'][employee_id][day]:
                        day_shift_counts[shift] += 1
                        day_total += 1

            total_staff_per_day.append(day_total)
            for ts in restaurant_data['time_slots']:
                shift_counts[ts].append(day_shift_counts[ts])

        # Print shift staffing analysis
        for i, ts in enumerate(restaurant_data['time_slots']):
            avg_staff = sum(shift_counts[ts]) / len(shift_counts[ts]) if shift_counts[ts] else 0
            min_staff = min(shift_counts[ts]) if shift_counts[ts] else 0
            max_staff = max(shift_counts[ts]) if shift_counts[ts] else 0
            req_staff = restaurant_data['min_staff'][i]

            print(f"  {ts}: Required={req_staff}, Avg={avg_staff:.1f}, Min={min_staff}, Max={max_staff}")

        # Print total daily staffing analysis
        avg_total = sum(total_staff_per_day) / len(total_staff_per_day) if total_staff_per_day else 0
        min_total = min(total_staff_per_day) if total_staff_per_day else 0
        max_total = max(total_staff_per_day) if total_staff_per_day else 0
        req_total = restaurant_data['min_daily_staff']

        print(f"\n  Total daily staff: Required={req_total}, Avg={avg_total:.1f}, Min={min_total}, Max={max_total}")

        return True
    else:
        print(f"Failed to create schedule even with relaxed constraints.")
        print("Please consider:")
        print("1. Adding more employees to your roster")
        print("2. Adjusting shift times or adding more shift options")
        print("3. Running a simplified version with fewer constraints")
        return False

if __name__ == "__main__":
    run_scheduler()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.3/277.3 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
Starting restaurant employee scheduler with 8-hour shifts
Using existing employee data with 25 employees
Creating schedule with 8-hour shifts:
  Time slots: ['04:00-12:00', '08:00-16:00', '12:00-20:00', '15:30-23:30']
  Minimum staff per shift: [3, 6, 6, 6]
  Minimum total staff per day: 16
  Labor laws: {'max_shifts_per_day': 1, 'min_rest_hours': 10, 'enforce_fairness': False}
This may take a few minutes...
Successfully loaded employee data with 25 employees
Solving scheduling problem...
No solution found. Status: 3
Initial scheduling attempt failed. Trying with relaxed constraints...
Successfully loaded employee data with 25 employees
Solving scheduling problem...
No soluti